In [1]:
!pip install transformers datasets

In [2]:
import math
import time
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from tensorflow import keras
from torch.utils.data import DataLoader, Dataset
from transformers import logging
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, DistilBertTokenizer, DistilBertModel, AutoTokenizer
from transformers import DefaultDataCollator, TFDistilBertForSequenceClassification
from transformers import DistilBertTokenizerFast, DataCollatorWithPadding
from datasets import load_dataset
import warnings
warnings.filterwarnings('ignore')

# Controlling the pseudo-randomness.
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [3]:
df = pd.read_csv('train_16project.csv')
df.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [4]:
from sklearn.model_selection import train_test_split

# First split: Divide into training (80%) and testing (20%)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Second split: Divide training data into training (80%) and validation (20%)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)


In [5]:
# Initialize the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Function to tokenize the data
def tokenize_data(df, tokenizer, max_length=512):
    return tokenizer(list(df['full_text']), max_length=max_length, truncation=True, padding="max_length", return_tensors="pt")

# Tokenizing the training and validation data
train_encodings = tokenize_data(train_df, tokenizer)
val_encodings = tokenize_data(val_df, tokenizer)
test_encodings = tokenize_data(test_df, tokenizer)

In [6]:
from transformers import DistilBertForSequenceClassification, DistilBertConfig

# Configuring the model to output 6 continuous values
config = DistilBertConfig.from_pretrained('distilbert-base-uncased')
config.num_labels = 6  # We need to output 6 scores per text
model = DistilBertForSequenceClassification(config)

# Modifying the classifier to a linear regression output
model.classifier = torch.nn.Linear(config.dim, 6)  # `config.dim` is the size of the hidden states


In [7]:
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW, get_scheduler
from tqdm import tqdm

# Prepare the dataset for PyTorch
train_labels = torch.tensor(train_df[['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']].values, dtype=torch.float)
val_labels = torch.tensor(val_df[['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']].values, dtype=torch.float)
test_labels = torch.tensor(test_df[['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']].values, dtype=torch.float)

train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], val_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

# Preparing data loaders, model, and optimizer as previously described
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

In [8]:
num_epochs = 10
# Custom accuracy function
def custom_accuracy(y_true, y_pred):
    return torch.mean(((y_true - y_pred).abs() <= 0.25).float()).item()

# Setup model, optimizer, and scheduler

optimizer = AdamW(model.parameters(), lr= 5e-5)
scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_loader) * num_epochs
)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training, Validation, and Testing loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    train_accuracy = []
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        b_input_ids, b_input_mask, b_labels = [t.to(device) for t in batch]
        outputs = model(b_input_ids, attention_mask=b_input_mask)
        loss = torch.nn.MSELoss()(outputs.logits, b_labels)
        acc = custom_accuracy(b_labels, outputs.logits)
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        train_loss += loss.item()
        train_accuracy.append(acc)

    # Validation step
    model.eval()
    val_accuracy = []
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            b_input_ids, b_input_mask, b_labels = [t.to(device) for t in batch]
            outputs = model(b_input_ids, attention_mask=b_input_mask)
            val_loss = torch.nn.MSELoss()(outputs.logits, b_labels)
            acc = custom_accuracy(b_labels, outputs.logits)
            val_accuracy.append(acc)
            val_loss += val_loss.item()

    print(f"Epoch {epoch+1}: Train Loss = {train_loss / len(train_loader)}; Train Accuracy = {sum(train_accuracy)/len(train_accuracy)}; Val Accuracy = {sum(val_accuracy)/len(val_accuracy)}; Validation Loss = {val_loss / len(val_loader)}")

# Testing loop
test_predictions = []
test_actuals = []
test_loss = 0
test_accuracy = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing"):
        b_input_ids, b_input_mask, b_labels = [t.to(device) for t in batch]
        outputs = model(b_input_ids, attention_mask=b_input_mask)
        test_loss = torch.nn.MSELoss()(outputs.logits, b_labels)
        acc = custom_accuracy(b_labels, outputs.logits)
        test_predictions.extend(outputs.logits.cpu().numpy())
        test_actuals.extend(b_labels.cpu().numpy())
        test_accuracy.append(acc)
        test_loss += test_loss.item()


# Create DataFrame to compare predictions with actuals
predictions_df = pd.DataFrame(test_predictions, columns=['pred_cohesion', 'pred_syntax', 'pred_vocabulary', 'pred_phraseology', 'pred_grammar', 'pred_conventions'])
actuals_df = pd.DataFrame(test_actuals, columns=['true_cohesion', 'true_syntax', 'true_vocabulary', 'true_phraseology', 'true_grammar', 'true_conventions'])
results_df = pd.concat([actuals_df, predictions_df], axis=1)
print(results_df.head())

print(f"Test Loss = {test_loss / len(test_loader)}; Test Accuracy = {sum(test_accuracy)/len(test_accuracy)}")


Validating: 100%|██████████| 40/40 [00:02<00:00, 17.56it/s]


Epoch 1: Train Loss = 0.7176156873535958; Train Accuracy = 0.2597753090522946; Val Accuracy = 0.2614583410322666; Validation Loss = 0.02430631034076214


Validating: 100%|██████████| 40/40 [00:02<00:00, 17.67it/s]


Epoch 2: Train Loss = 0.43954026006209623; Train Accuracy = 0.29129070254742717; Val Accuracy = 0.332812511920929; Validation Loss = 0.02292693965137005


Validating: 100%|██████████| 40/40 [00:02<00:00, 17.66it/s]


...

Epoch 29: Train Loss = 0.07054018691941431; Train Accuracy = 0.6597664735879109; Val Accuracy = 0.3153645921498537; Validation Loss = 0.006642424967139959


Validating: 100%|██████████| 40/40 [00:02<00:00, 17.60it/s]


Epoch 30: Train Loss = 0.06872155208876178; Train Accuracy = 0.6594789635603595; Val Accuracy = 0.3127604264765978; Validation Loss = 0.006515701301395893


Testing: 100%|██████████| 49/49 [00:02<00:00, 17.23it/s]

   true_cohesion  true_syntax  true_vocabulary  true_phraseology  \
0            3.0          2.5              2.5               2.0   
1            3.0          2.0              3.0               3.5   
2            4.0          4.0              3.0               4.0   
3            3.0          3.0              3.5               3.0   
4            3.5          3.5              3.5               3.5   

   true_grammar  true_conventions  pred_cohesion  pred_syntax  \
0           2.0               2.0       2.583694     3.179252   
1           3.0               3.0       2.578758     2.604294   
2           4.0               4.0       3.813802     2.885073   
3           3.5               3.5       2.660313     2.904942   
4           3.0               3.5       3.603974     3.403027   

   pred_vocabulary  pred_phraseology  pred_grammar  pred_conventions  
0         3.240652          3.256781      3.038039          2.968935  
1         2.637927          2.458733      2.888671        

In [9]:
test_ids = test_df['text_id'].values  # Extract the IDs for later use

# Convert the test data into the format expected by the model
test_encodings = tokenize_data(test_df, tokenizer)
test_input_ids = test_encodings['input_ids']
test_attention_mask = test_encodings['attention_mask']
test_labels = torch.tensor(test_df[['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']].values, dtype=torch.float)

# Model evaluation setup
model.eval()
test_predictions = []
test_losses = []
test_accuracies = []
with torch.no_grad():
    for idx in range(len(test_df)):
        # Extract inputs for the current test instance
        input_ids = test_input_ids[idx].unsqueeze(0).to(device)  # Add batch dimension
        attention_mask = test_attention_mask[idx].unsqueeze(0).to(device)  # Add batch dimension
        labels = test_labels[idx].unsqueeze(0).to(device)  # Add batch dimension for labels
        outputs = model(input_ids, attention_mask=attention_mask)

        # Calculate loss and accuracy
        loss = torch.nn.MSELoss()(outputs.logits, labels)
        acc = custom_accuracy(labels, outputs.logits)

        # Store predictions, loss, and accuracy
        test_predictions.append(outputs.logits.squeeze(0).cpu().numpy())  # Remove batch dimension and move to CPU
        test_losses.append(loss.item())
        test_accuracies.append(acc)

# Calculate average loss and accuracy
average_test_loss = sum(test_losses) / len(test_losses)
average_test_accuracy = sum(test_accuracies) / len(test_accuracies)

# Create DataFrame to compare predictions with IDs
predictions_df = pd.DataFrame(test_predictions, columns=['pred_cohesion', 'pred_syntax', 'pred_vocabulary', 'pred_phraseology', 'pred_grammar', 'pred_conventions'])
ids_df = pd.DataFrame(test_ids, columns=['text_id'])
results_df = pd.concat([ids_df, predictions_df], axis=1)
print(results_df.head())
print(f"Average Test Loss: {average_test_loss}")
print(f"Average Test Accuracy: {average_test_accuracy}")

        text_id  pred_cohesion  pred_syntax  pred_vocabulary  \
0  772D27D400BB       2.583693     3.179252         3.240652   
1  9E8F3C6405CA       2.578758     2.604295         2.637927   
2  948771F795EB       3.813803     2.885074         3.293267   
3  FE14D7378CFB       2.660312     2.904941         3.240855   
4  7AAE019F70D6       3.603974     3.403028         3.447347   

   pred_phraseology  pred_grammar  pred_conventions  
0          3.256783      3.038040          2.968935  
1          2.458734      2.888670          3.390501  
2          3.299035      2.712234          2.830544  
3          2.910870      2.729234          2.319380  
4          3.079239      3.113676          3.108289  
Average Test Loss: 0.38734501256877474
Average Test Accuracy: 0.31587910366697786


In [10]:
merged_df = pd.merge(test_df, results_df, on='text_id', how='left')
merged_df.to_csv('distilBERT_test_predictions.csv', index=False)

In [11]:
merged_df.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,pred_cohesion,pred_syntax,pred_vocabulary,pred_phraseology,pred_grammar,pred_conventions
0,772D27D400BB,It god to have a possitive attitude when you d...,3.0,2.5,2.5,2.0,2.0,2.0,2.583693,3.179252,3.240652,3.256783,3.038040,2.968935
1,9E8F3C6405CA,Why do people ask more then one person for adv...,3.0,2.0,3.0,3.5,3.0,3.0,2.578758,2.604295,2.637927,2.458734,2.888670,3.390501
2,948771F795EB,"We accomplish more when we are active, and are...",4.0,4.0,3.0,4.0,4.0,4.0,3.813803,2.885074,3.293267,3.299035,2.712234,2.830544
3,FE14D7378CFB,Do you agree or disagree about imagination bei...,3.0,3.0,3.5,3.0,3.5,3.5,2.660312,2.904941,3.240855,2.910870,2.729234,2.319380
4,7AAE019F70D6,I disagree with the principal saying that all ...,3.5,3.5,3.5,3.5,3.0,3.5,3.603974,3.403028,3.447347,3.079239,3.113676,3.108289


In [12]:
results_df.describe()

,pred_cohesion,pred_syntax,pred_vocabulary,pred_phraseology,pred_grammar,pred_conventions
count,783.000000,783.000000,783.000000,783.000000,783.000000,783.000000
mean,3.159366,3.078925,3.282201,3.111620,3.027177,3.077715
std,0.568206,0.518351,0.492014,0.530133,0.591721,0.571564
min,1.371086,1.692512,1.906839,1.569690,1.525700,1.369442
25%,2.776169,2.711581,2.916236,2.731097,2.569701,2.697888
50%,3.163273,3.096978,3.296294,3.121753,3.009525,3.091959
75%,3.553957,3.442828,3.616905,3.491226,3.470718,3.466598
max,4.670754,4.485359,5.219347,4.580180,4.569763,4.696817


In [13]:
test_df.describe()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
count,783.000000,783.000000,783.000000,783.000000,783.000000,783.000000
mean,3.126437,3.015964,3.212644,3.109195,3.038953,3.077267
std,0.651069,0.651626,0.574217,0.658812,0.696687,0.680450
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2.500000,2.500000,3.000000,2.500000,2.500000,2.500000
50%,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
75%,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000
max,5.000000,4.500000,5.000000,5.000000,5.000000,5.000000
